<a href="https://colab.research.google.com/github/ritikraj660/Speaker-recognition/blob/master/Project_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install spafe
!pip install soundfile

     |████████████████████████████████| 81kB 2.1MB/s 


In [0]:
import librosa
import soundfile
import tqdm
from scipy.io import wavfile
import glob
import os,pickle
import numpy as np
from spafe.features.mfcc import mfcc, imfcc
import csv
import pandas as pd
from tqdm import tqdm

# Preprocessing

In [0]:
"""import scipy.io.wavfile
import spafe.utils.vis as vis


x=np.array(x)
y=np.array(y)
result = np.column_stack((x,y))
import csv
with open('people.csv', 'a') as peopleFile:
    writer = csv.writer(peopleFile)
    writer.writerows(result)
csvFile.close()"""

"import scipy.io.wavfile\nimport spafe.utils.vis as vis\n\n\nx=np.array(x)\ny=np.array(y)\nresult = np.column_stack((x,y))\nimport csv\nwith open('people.csv', 'a') as peopleFile:\n    writer = csv.writer(peopleFile)\n    writer.writerows(result)\ncsvFile.close()"

# Feature Extraction :

In [0]:
#f = pd.DataFrame()
def tocsv(x,y):#take numpy 2d array and append it with 
    #df2=pd.DataFrame(x)
    #f = pd.concat([f, df2])
    k=[]
    m=len(x)
    #print(m)
    for i in range(m):
        k.append(y)
    y=np.array(k).T    
    result = np.column_stack((x,y))    
    import csv
    with open('people.csv', 'a') as peopleFile:
        writer = csv.writer(peopleFile)
        writer.writerows(result)
    peopleFile.close()
#f.to_csv('mfcc.csv')    

In [0]:
def mfcc_(sig,fs):
    mfccs  = mfcc(sig, fs=fs, num_ceps=20, pre_emph=0, pre_emph_coeff=0.95, 
              win_len=0.025, win_hop=0.01, win_type='hamming', nfilts=26, 
              nfft=512, low_freq=None, high_freq=None, scale='constant', 
              dct_type=2, use_energy=False, lifter=22, normalize=1)
    return mfccs

In [0]:
def s2c():
    x,y=[],[]
    count =0
    for file in tqdm(glob.iglob('G:\LibriSpeech\dev-clean\\**/*.flac', recursive=True)):
        file_name=os.path.basename(file)
        user = file_name.split("-")[0]
        #print(f)
        with soundfile.SoundFile(file) as sound_file:
                signal = sound_file.read(dtype="float32")
                sr=sound_file.samplerate
        
        
        feature = mfcc_(signal,sr)
        #x.append(feature)
        #y.append(user)
        tocsv(feature,user)
        """count+=1
        if count>=10:
            break"""
    """      
    x=np.array(x)
    y=np.array(y)
    result = np.column_stack((x,y))
    """
    #return x,y

In [0]:
s2c()

0it [00:00, ?it/s]


# Test :

In [0]:
def m5(num_classes=5):
    print('Using Model M5')
    m = Sequential()
    m.add(Conv1D(128,
                 input_shape=[AUDIO_LENGTH, 1],
                 kernel_size=80,
                 strides=4,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(128,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(512,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Lambda(lambda x: K.mean(x, axis=1)))  # Same as GAP for 1D Conv Layer
    m.add(Dense(num_classes, activation='softmax'))
    return m


def get_data(file_list):
    def load_into(_filename, _x, _y):
        with open(_filename, 'rb') as f:
            audio_element = pickle.load(f)
            _x.append(audio_element['audio'])
            _y.append(int(audio_element['class_id']))

    x, y = [], []
    for filename in file_list:
        load_into(filename, x, y)
    return np.array(x), np.array(y)


if __name__ == '__main__':

    num_classes = 5
    model = m5(num_classes=num_classes)

    if model is None:
        exit('Something went wrong!!')

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print(model.summary())

    train_files = glob(os.path.join(OUTPUT_DIR_TRAIN, '**.pkl'))
    x_tr, y_tr = get_data(train_files)
    y_tr = to_categorical(y_tr, num_classes=num_classes)

    test_files = glob(os.path.join(OUTPUT_DIR_TEST, '**.pkl'))
    x_te, y_te = get_data(test_files)
    y_te = to_categorical(y_te, num_classes=num_classes)

    print('x_tr.shape =', x_tr.shape)
    print('y_tr.shape =', y_tr.shape)
    print('x_te.shape =', x_te.shape)
    print('y_te.shape =', y_te.shape)

    # if the accuracy does not increase over 10 epochs, reduce the learning rate by half.
    reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
    batch_size = 128
    model.fit(x=x_tr,
              y=y_tr,
              batch_size=batch_size,
              epochs=400,
              verbose=1,
              shuffle=True,
              validation_data=(x_te, y_te),
              callbacks=[reduce_lr])

In [0]:
x = df3.

# Google Drive Data access

In [0]:
#!pip install -U -q PyDrive
#https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1njkTiNjPtcLZx0V5zr-qWXAEXrQl45PW'

In [28]:
fluff, id = link.split('=')
print (id) 

1njkTiNjPtcLZx0V5zr-qWXAEXrQl45PW


In [0]:
import pandas as pd
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('people.csv')  
df3 = pd.read_csv('people.csv')

In [30]:
from sklearn.utils import shuffle
df3 = shuffle(df3)
df3.head()

,-1.6153728139728258,-0.566095886747293,-2.2038676823804004,0.2065565102352862,0.26361972118538163,1.1277980364132403,-0.3869602504543645,2.193784609773862,0.9997128846170393,0.9399494391010893,0.6090782488017761,-1.6818174423852228,0.5630336280377202,0.3519422417266513,0.921815469376084,-0.27177671449756524,-0.16286144968094976,0.20477036437616464,0.6802467426860398,0.16774122569148825,1272
880977,0.449591,0.625241,0.666808,-0.066148,0.715964,-0.219909,-2.889133,-0.126205,-0.517977,0.011492,0.863159,-0.617024,0.070080,2.155270,-0.466508,-0.583316,0.284502,-0.296485,0.063797,-0.076893,3170
971905,0.207373,1.183828,0.017974,-0.019162,-0.335244,-0.165906,-0.595883,-0.610589,0.191852,1.158782,-1.476227,-1.471006,1.290293,-0.683375,-1.086941,0.382902,-0.585269,-0.318746,0.231989,-0.356906,3576
1655817,-0.119851,1.746556,0.827917,-0.732080,-1.015105,-1.077252,0.363192,-0.973797,-0.493972,-0.391186,-0.248488,-0.781056,0.279936,-1.058551,0.698711,0.533189,0.018126,-0.494418,0.068030,-0.446701,777
84410,1.331068,0.752078,-1.864311,-0.858137,-0.500464,-0.853370,-1.820106,1.674754,1.850190,-1.226498,-0.462252,-0.343982,-0.675220,-1.457112,-0.486021,-0.880672,0.197291,-0.143711,-1.150095,-0.403232,1462
331916,1.286193,0.753882,-2.506486,-0.486509,-0.195666,-0.103662,0.271168,-0.926106,-0.238654,-0.807709,-1.717898,0.424991,-0.585218,-0.315617,-1.830477,-0.250803,-0.048306,-0.508517,-0.242160,-0.440821,1993


In [0]:
x = df3.drop(columns="1272")
y = df3["1272"]

In [0]:
print(x.shape,y.shape)

(1933635, 20) (1933635,)


In [32]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(1546908, 20) (1546908,)
(386727, 20) (386727,)


In [0]:
#print(y_train.value_counts(),y_test.value_counts())

In [0]:
from sklearn.svm import LinearSVR
#svc_model = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#    decision_function_shape='ovr', degree=3, gamma='scale',
#    kernel='rbf', max_iter=-1, probability=False, random_state=None,
#    shrinking=True, tol=0.001, verbose=False)

svc_model = LinearSVR(epsilon=0.0, tol=0.0001, C=1.0, loss='epsilon_insensitive', 
          fit_intercept=True, intercept_scaling=1.0, dual=True, verbose=0, 
          random_state=None, max_iter=1000)

In [0]:
svc_model.fit(X_train,y_train)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [0]:
y_predict = svc_model.predict(X_test)

In [0]:
"""from sklearn.metrics import accuracy_score

#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_predict)
#DataFlair - Print the accuracy
print((accuracy*100))"""

'from sklearn.metrics import accuracy_score\n\n#DataFlair - Calculate the accuracy of our model\naccuracy=accuracy_score(y_true=y_test, y_pred=y_predict)\n#DataFlair - Print the accuracy\nprint((accuracy*100))'

In [0]:
from sklearn.metrics import mean_squared_error

In [0]:
mean_squared_error(y_test,y_predict)

6356161.995744575

In [0]:
svc_model.score(X_test,y_test)

-0.07208669197378859

# MLP classifiers :

In [0]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
model=MLPClassifier(hidden_layer_sizes=(128,128,128), activation='tanh', solver='adam', 
                    alpha=0.0001, batch_size=64, learning_rate='constant', 
                    learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
                    random_state=None, tol=0.0001, verbose=10, warm_start=False, 
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
                    validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
                    n_iter_no_change=10)

In [45]:
#DataFlair - Train the model
model.fit(X_train,y_train)
#model1.fit(x_train,y_train)

Iteration 1, loss = 2.32092332
Iteration 2, loss = 2.01885616
Iteration 3, loss = 1.95502957
Iteration 4, loss = 1.92167984
Iteration 5, loss = 1.90138179
Iteration 6, loss = 1.88637682
Iteration 7, loss = 1.87584672
Iteration 8, loss = 1.86692189
Iteration 9, loss = 1.86068058
Iteration 10, loss = 1.85501761
Iteration 11, loss = 1.84982203
Iteration 12, loss = 1.84692820
Iteration 13, loss = 1.84371616
Iteration 14, loss = 1.83959222
Iteration 15, loss = 1.83696868
Iteration 16, loss = 1.83474547
Iteration 17, loss = 1.83290447
Iteration 18, loss = 1.83086505
Iteration 19, loss = 1.82931085
Iteration 20, loss = 1.82753197
Iteration 21, loss = 1.82601201
Iteration 22, loss = 1.82429944
Iteration 23, loss = 1.82300024
Iteration 24, loss = 1.82181541
Iteration 25, loss = 1.82027251
Iteration 26, loss = 1.81960707
Iteration 27, loss = 1.81834662
Iteration 28, loss = 1.81741764
Iteration 29, loss = 1.81606164
Iteration 30, loss = 1.81537828
Iteration 31, loss = 1.81382399
Iteration 32, los

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(activation='tanh', alpha=0.0001, batch_size=64, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(128, 128, 128), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=10, warm_start=False)

In [0]:
#DataFlair - Predict for the test set
from tqdm import tqdm
y_pred=model.predict(X_test)
y_prob = model.predict_proba(X_test)
#y_pred1=model1.predict(x_test)

In [47]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 49.04%


In [0]:
import pickle 
  
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(model) 